In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

## US Congressional Voting Records Dataset

In [3]:
# Loading voting dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', \
                    names=['party', 'infants', 'water', 'budget', 'physician', 'salvador', \
                           'religious', 'satellite', 'aid', 'missile', 'immigration', 'synfuels', \
                           'education', 'superfund', 'crime', 'duty_free_exports', 'eaa_rsa' \
                          ])
df_copy = df.copy()

# Binarize votes
df.replace(['n', '?'], 0, inplace=True)
df.replace('y', 1, inplace=True)
df.head()

,party,infants,water,budget,physician,salvador,religious,satellite,aid,missile,immigration,synfuels,education,superfund,crime,duty_free_exports,eaa_rsa
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,0
2,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


## KNN

In [4]:
# Create arrays for the features and the response variable
y = df['party'].values
X = df.drop('party', axis=1).values

In [5]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Create a k-NN classifier with 6 neighbors
knn = KNeighborsClassifier(n_neighbors=6)

In [7]:
# Fit the classifier to the data
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [8]:
# Predict the labels for the training data X
y_pred = knn.predict(X_train)

In [9]:
# Predict and print the label for the new data point X_new
new_prediction = knn.predict(X_test)

## Accuracy

In [10]:
# Accuracy on training set
accuracy_score(y_pred, y_train)

0.9396551724137931

In [11]:
# Accuracy on testing set
accuracy_score(new_prediction, y_test)

0.9655172413793104

## Dropping missing data

In [12]:
# Restore original dataframe
df = df_copy
df.head()

,party,infants,water,budget,physician,salvador,religious,satellite,aid,missile,immigration,synfuels,education,superfund,crime,duty_free_exports,eaa_rsa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [13]:
# Datasets encode missing data in different ways. It may be a '9999', other times a 0 or '?'
# Convert '?' to NaN
df[df == '?'] = np.nan
df.head()

,party,infants,water,budget,physician,salvador,religious,satellite,aid,missile,immigration,synfuels,education,superfund,crime,duty_free_exports,eaa_rsa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [14]:
# Binarize votes
df[df == 'n'] = 0
df[df == 'y'] = 1
df.head()

,party,infants,water,budget,physician,salvador,religious,satellite,aid,missile,immigration,synfuels,education,superfund,crime,duty_free_exports,eaa_rsa
0,republican,0,1,0,1,1,1,0,0,0,1,NaN,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,NaN
2,democrat,NaN,1,1,NaN,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,NaN,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,NaN,1,1,1,1


In [15]:
# Print the number of NaNs
df.isnull().sum()

party                  0
infants               12
water                 48
budget                11
physician             11
salvador              15
religious             11
satellite             14
aid                   15
missile               22
immigration            7
synfuels              21
education             31
superfund             25
crime                 17
duty_free_exports     28
eaa_rsa              104
dtype: int64

In [16]:
# Print shape of original DataFrame
df.shape

(435, 17)

In [17]:
# Drop missing values and print shape of new DataFrame
df.dropna().shape

(232, 17)

## Imputing missing data in Pipeline

In [30]:
# Setup the Imputation transformer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [32]:
# Instantiate the KNN classifier
knn = KNeighborsClassifier()

In [33]:
# Setup the pipeline with the required steps
steps = [('imputation', imp),
        ('KNN', knn)]

In [34]:
# Create the pipeline
pipeline = Pipeline(steps)

In [35]:
# Create arrays for the features and the response variable
y = df['party']
X = df.drop('party', axis=1)

In [36]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [37]:
# Fit the pipeline to the train set
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('imputation',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('KNN',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

In [38]:
# Predict the labels of the test set
y_pred = pipeline.predict(X_test)

In [39]:
# Compute metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    democrat       0.95      0.95      0.95        76
  republican       0.93      0.93      0.93        55

    accuracy                           0.94       131
   macro avg       0.94      0.94      0.94       131
weighted avg       0.94      0.94      0.94       131

